### Put data into csv

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #, force_remount=True

Mounted at /content/drive


In [ ]:
import os

class song:
    def __init__(self, title, lyrics):
        self.title = title
        self.lyrics = lyrics

def get_file_content(file_path):
  with open(file_path) as f:
    contents = f.readlines()
    return contents

def get_all_files(data, dir_path):
  if dir_path.endswith('.txt'):
    d = song(dir_path.split("/")[-1], get_file_content(dir_path))
    data.append(d)
    return data
    
  if dir_path.endswith('.DS_Store'):
    return data
  
  directory_files = os.listdir(dir_path)
  for dir in directory_files:
    dir = dir_path + "/" + dir
    get_all_files(data, dir)
  return data
  
data = []
data = get_all_files(data, '/content/drive/Shareddrives/AdventuresInNLP/Project/yearwise_dataset')

In [ ]:
len(data)

10229

In [ ]:
lyrics = []
titles = []
for i in range(len(data)):
  titles.append(data[i].title)
  lyric = "".join(data[i].lyrics)
  lyric = lyric.replace("\n ", "")
  lyric = lyric.replace("\n","",1)
  lyrics.append(lyric)
  

In [ ]:
df_data = {
   'lyrics': lyrics,
   'title': titles
}
df = pd.DataFrame(df_data)

In [ ]:
df.head()

,lyrics,title
0,tum agar mujhako na chaaho to koi baat nahi\nt...,1953 #_# Tum Agar Mujhko Na Chaho To #_# Raj K...
1,tum toh dil ke taar chhedakar ho gaye bekhabar...,1953 #_# Tum To Dil Ke Taar Chhed Kar #_# Tala...
2,tum jo hamaare mit na hote\ngeet ye mere geet ...,1953 #_# Tum Jo Hamare Meet Na Hote #_# Raj Ka...
3,dha sa\nni re\nsa ga\nni sa\n\ntum saaz chedoo...,1953 #_# Ulfut Ka Saaz Chhedo #_# Aurat.txt
4,ye zindagi usee kii he jo kisi kaa ho gaya\npy...,1953 #_# Yeh Zindagi Usiki Hai (Part - I) #_# ...


In [ ]:
df.to_csv('/content/drive/Shareddrives/AdventuresInNLP/Project/Bollywood_lyrics.csv')

### GPT

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #, force_remount=True

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.4 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [ ]:
# Load Hinglish song lyrics dataset
df = pd.read_csv("/content/drive/Shareddrives/AdventuresInNLP/Project/Bollywood_lyrics.csv")

In [ ]:
df.head()

,Unnamed: 0,lyrics,title
0,0,tum agar mujhako na chaaho to koi baat nahi\nt...,1953 #_# Tum Agar Mujhko Na Chaho To #_# Raj K...
1,1,tum toh dil ke taar chhedakar ho gaye bekhabar...,1953 #_# Tum To Dil Ke Taar Chhed Kar #_# Tala...
2,2,tum jo hamaare mit na hote\ngeet ye mere geet ...,1953 #_# Tum Jo Hamare Meet Na Hote #_# Raj Ka...
3,3,dha sa\nni re\nsa ga\nni sa\n\ntum saaz chedoo...,1953 #_# Ulfut Ka Saaz Chhedo #_# Aurat.txt
4,4,ye zindagi usee kii he jo kisi kaa ho gaya\npy...,1953 #_# Yeh Zindagi Usiki Hai (Part - I) #_# ...


In [ ]:
df.shape

(10229, 3)

In [ ]:
len(df['lyrics'][0].split(' '))

201

Create Test Set

In [ ]:
#Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
df = df[df['lyrics'].apply(lambda x: len(x.split(' ')) < 350)]

In [ ]:
df.shape

(9937, 3)

In [ ]:
#Create a very small test set to compare generated text with the reality
test_set = df.sample(n = 200)
df = df.loc[~df.index.isin(test_set.index)]

In [ ]:
#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

In [ ]:
#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_lyrics'] = test_set['lyrics'].str.split().str[-20:].apply(' '.join)
test_set['lyrics'] = test_set['lyrics'].str.split().str[:-20].apply(' '.join)

In [ ]:
test_set.head()

,index,Unnamed: 0,lyrics,title,True_end_lyrics
0,8940,8940,teri wo ek nazar chhu kar gayi is kadar khota ...,2013 #_# Ek Nazar #_# Daastan.txt,paau mahiya ve mar bhi naa paau mahiya ve karu...
1,8659,8659,jaane do jaane do mujhe jaana hai are waada jo...,2013 #_# Jaane De Mujhe Jaana Hai #_# Anand Ba...,saath aaye hairaangi hai aane de aane de mujhe...
2,6115,6115,rahegaa ishq teraa khaak me milaake mujhe hue ...,1966 #_# Aa Aa Bhi Ja #_# Teesri Kasam.txt,chaand chhupane chalaa ho aa aa bhi jaa raat d...
3,2024,2024,zindagi pyar ka geet hai ise har dil ko gana p...,1983 #_# Zindagi Pyar Ka Geet Hai- Female #_# ...,pyar ka geet hai ise har dil ko gana padega zi...
4,4195,4195,teri hai dua hai dua teri hai dua hai dua kais...,2009 #_# Hai Dua #_# Meherbaan.txt,dua teri hai dua mere sapno ko mil gaya rehnum...


In [ ]:
test_set['lyrics'][0]

'teri wo ek nazar chhu kar gayi is kadar khota hi ja rahaa mujhase meraa safar kaisaa hai ye nasha kaisaa khumaar hai khwabo kii hai zamin sirf teri pyaas hai dhundha karu ab raat din tere nishaan tere nishaan tere bin ji nahi paau mahiya mar bhi naa paau mahiya karu to mai kya karu mahiya jana mai jana tera hi khyal hai teri hi to hai phikr meri har baat pe tera hi to hai zikr mujhpe padey tera karm bas ab yahi hai intejhaar tere bin ji nahi paau mahiya mar bhi naa paau mahiya karu to mai kya karu mahiya jana mai jana manzile tu hi he tu hi to he rasta jaau to kya hua tere bin bataa mai kaha mil jaao naa ab tum mujhe puri ho ye daasta tere bin ji nahi paau mahiya ve mar bhi naa paau mahiya ve karu to mai kya karu mahiya ve janu naa tere bin ji nahi'

In [ ]:
test_set['True_end_lyrics'][0]

'paau mahiya ve mar bhi naa paau mahiya ve karu to mai kya karu mahiya ve janu naa janu naa'

Create the dataset

In [ ]:
class SongLyrics:  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['lyrics']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
    
dataset = SongLyrics(df['lyrics'], truncate=True, gpt2_type="gpt2")

Get the tokenizer & model

In [ ]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
model = train(dataset, model, tokenizer)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


9737it [19:58,  8.12it/s]


Training epoch 1
tensor(1.5904, device='cuda:0', grad_fn=<NllLossBackward0>)


9737it [19:57,  8.13it/s]


...

9737it [19:58,  8.12it/s]


Training epoch 3
tensor(1.6045, device='cuda:0', grad_fn=<NllLossBackward0>)


9737it [20:00,  8.11it/s]


Training epoch 4
tensor(1.4580, device='cuda:0', grad_fn=<NllLossBackward0>)


9737it [19:56,  8.14it/s]


In [ ]:
torch.save(model.state_dict(), "/content/drive/Shareddrives/AdventuresInNLP/Project/gpt2-5-epochs")

In [ ]:
model = model.cuda()
model.load_state_dict(torch.load("/content/drive/Shareddrives/AdventuresInNLP/Project/gpt2-5-epochs"))

<All keys matched successfully>

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(197, len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['lyrics'][i], entry_count=1)
    print("Generating lyrics for i = ", i )
    print(x)
    print("=" * 20)
    generated_lyrics.append(x)
  return generated_lyrics

In [ ]:
test_set.shape

(200, 5)

In [ ]:
test_set.head()

,index,Unnamed: 0,lyrics,title,True_end_lyrics
0,8940,8940,teri wo ek nazar chhu kar gayi is kadar khota ...,2013 #_# Ek Nazar #_# Daastan.txt,paau mahiya ve mar bhi naa paau mahiya ve karu...
1,8659,8659,jaane do jaane do mujhe jaana hai are waada jo...,2013 #_# Jaane De Mujhe Jaana Hai #_# Anand Ba...,saath aaye hairaangi hai aane de aane de mujhe...
2,6115,6115,rahegaa ishq teraa khaak me milaake mujhe hue ...,1966 #_# Aa Aa Bhi Ja #_# Teesri Kasam.txt,chaand chhupane chalaa ho aa aa bhi jaa raat d...
3,2024,2024,zindagi pyar ka geet hai ise har dil ko gana p...,1983 #_# Zindagi Pyar Ka Geet Hai- Female #_# ...,pyar ka geet hai ise har dil ko gana padega zi...
4,4195,4195,teri hai dua hai dua teri hai dua hai dua kais...,2009 #_# Hai Dua #_# Meherbaan.txt,dua teri hai dua mere sapno ko mil gaya rehnum...


In [ ]:
smaller_test_set = test_set[197:200]
smaller_test_set

,index,Unnamed: 0,lyrics,title,True_end_lyrics
197,5170,5170,yuhi nehi tujhpe dil ye fida hai sabse tu aled...,2017 #_# Dil Mera Chahe #_# Dil Mera Chahe.txt,jabhi tu aae tujhse me kahedoo wapas na jana b...
198,9605,9605,terimeri meriteri prem kahani hai mushqil do l...,2012 #_# Teri Meri (Bodyguard) #_# Bollywood H...,mein yeh bayaan na ho paaye terimeri meriteri ...
199,4230,4230,tikhi tikhi nukili si bunde behake behake se b...,2009 #_# Bheeg Gaya Mera Mann (Cherrapunjee) #...,barase re ambar ka paani jisako pi pike dharat...


In [ ]:
test_set['lyrics'][198]

'terimeri meriteri prem kahani hai mushqil do lafzoon mein yeh bayaan na ho paaye ik ladka or ik ladki ki yeh kahani hai nayi do lafzoon mein yeh bayaan na ho paaye terimeri meriteri prem kahani hai mushqil do lafzoon mein yeh bayaan na ho paaye ik dooje se huey juda jab ik dooje ke liye bane terimeri meriteri prem kahani hai mushqil do lafzoon mein yeh bayaan na ho paaye tumse dil jo lagaya toh jahaan maine paaya kabhi socha na tha yeh meelon door hoga saaya kyun khuda tune mujhe aysa khwaab dikhaya jab haqiqat mein usey tordna tha ik dooje se huey juda jab ik dooje ke liye bane terimeri meriteri prem kahani hai mushqil do lafzoon mein yeh bayaan na ho paaye terimeri baaton ka har lamha sab se anjaana do lafzoon mein yeh bayaan na ho paaye har ehsaas mein tu hai har ik yaad mein tera afsaana do lafzoon mein yeh bayaan na ho paaye sara din beet jaaye saari raat jagaye bas khayal tumhara lamha lamha tadpaye yeh tadap keh rahi hai mitt jaaye faasle yeh tere mere darmayaan jo hain saare i

In [ ]:
#Run the functions to generate the lyrics

generated_lyrics = text_generation(test_set)
print(generated_lyrics)

100%|██████████| 1/1 [01:26<00:00, 86.47s/it]


Generating lyrics for i =  197
['yuhi nehi tujhpe dil ye fida hai sabse tu aledaa sabse jooda hai na mumkeen tujhsa koi chehara mil panaadil mera chahe jabhi tu aae tujhse me kahedoo wapas na jana baho me teri sari sab jaagu aakho se dekhu subhoka aana dil mera chahe jabhi tu aae tujhse me kahedoo wapas na jana baho me teri sari sab jaagu aakho se dekhu subhoka aana miloge kabhi jo tum batayenge kya ho tum milo aab akele raha jae na aaaaamiloge kabhi jo tum batayenge kya ho tum milo aab akele raha jae na bahat hi jaroori ho khatam sari doori ho koi faasaab saha jaena muskil hai dil ko bin tere samjhanadil mera chahe jabhi tu aae tujhse me kahedoo wapas na jana baho me teri sari sab jaagu aakho se dekhu subhoka aana dil mera chahe jabhi tu aae tujhse me kahedoo wapas na jana baho me teri sari sab jaagu aakho se dekhu subhoka aanaaaaaaaajab dil ki halat ho jo teri jiaarat hoo najar ko najara koi bhae na aaaaaajab dil ki halat ho jo teri jiaarat hoo najar ko najara koi bhae na yebo bekaar

100%|██████████| 1/1 [01:24<00:00, 84.10s/it]


Generating lyrics for i =  198
['terimeri meriteri prem kahani hai mushqil do lafzoon mein yeh bayaan na ho paaye ik ladka or ik ladki ki yeh kahani hai nayi do lafzoon mein yeh bayaan na ho paaye terimeri meriteri prem kahani hai mushqil do lafzoon mein yeh bayaan na ho paaye ik dooje se huey juda jab ik dooje ke liye bane terimeri meriteri prem kahani hai mushqil do lafzoon mein yeh bayaan na ho paaye tumse dil jo lagaya toh jahaan maine paaya kabhi socha na tha yeh meelon door hoga saaya kyun khuda tune mujhe aysa khwaab dikhaya jab haqiqat mein usey tordna tha ik dooje se huey juda jab ik dooje ke liye bane terimeri meriteri prem kahani hai mushqil do lafzoon mein yeh bayaan na ho paaye terimeri baaton ka har lamha sab se anjaana do lafzoon mein yeh bayaan na ho paaye har ehsaas mein tu hai har ik yaad mein tera afsaana do lafzoon mein yeh bayaan na ho paaye sara din beet jaaye saari raat jagaye bas khayal tumhara lamha lamha tadpaye yeh tadap keh rahi hai mitt jaaye faasle yeh ter

100%|██████████| 1/1 [00:41<00:00, 41.10s/it]

Generating lyrics for i =  199
['tikhi tikhi nukili si bunde behake behake se baadal uninde geet gaati hawa mein gungunaati ghata mein bheeg gaya mera mann ho mastiyon ke ghunt pi shokhiyon mein tair ja ishq ki galiyon mein aa inn palon mein thehar ja rab ka hai yeh aaina shakl haan isako dikha jindagi ghud daud hai do ghadi le le maja chamake chamake yeh jharano ke dhaare tan pe malamal si padati phuaare ped hai manchale se patte hai chulbule se bheeg gaya mera mann ho dagmagaati chaandani has rahi hai josh mein aur patange ga rahe haan raag maal kos mein banake naache behaya besahram pagali hawa jindagi milake gale has rahi de de duwaan barase pahi rahi mein kamaaht ke aale mein badi hejha kahara istana mein bah<|endoftext|>']
[['yuhi nehi tujhpe dil ye fida hai sabse tu aledaa sabse jooda hai na mumkeen tujhsa koi chehara mil panaadil mera chahe jabhi tu aae tujhse me kahedoo wapas na jana baho me teri sari sab jaagu aakho se dekhu subhoka aana dil mera chahe jabhi tu aae tujhse me 